Packages Used

-scikit allel (conda install -c conda-forge scikit-allel, pip install scikit-allel
              pip install scikit-allel[full] installs the  optional third-party packages
              including scipy, matplotlib, seaborn, pandas, scikit-learn, h5py and zarr)
           
-UMAP (conda install -c conda-forge umap-learn or pip install umap-learn)

-numpy, scikit-sklearn, pandas 

Plotting Libraries

    -Plotly (pip install plotly )

    -cufflinks (pip install cufflinks, integrates plotly and pandas dataframes)


In [2]:
import allel
allel.__version__

'1.2.0'

In [5]:
import os
import time
os.listdir('.')

['ALL.wgs.nhgri_coriell_affy_6.20140825.genotypes_has_ped.vcf',
 'ALL.wgs.nhgri_coriell_affy_6.20140825.genotypes_has_ped.vcf.gz',
 'affy_samples.20141118.panel',
 'Task.ipynb',
 '20131219.populations.tsv',
 '.ipynb_checkpoints']

In [6]:
# Could not directly load from a pandas dataframe because it currently does not return calldata/GT field.
#There's a PR though

data_loading_time=time.time()
data=allel.read_vcf('ALL.wgs.nhgri_coriell_affy_6.20140825.genotypes_has_ped.vcf',fields=['CHROM','GT','samples'])
print('TIME TAKEN FOR DATA LOADING: ',time.time()-data_loading_time)

TIME TAKEN FOR DATA LOADING:  205.68490982055664


In [7]:
data # Have loaded only the relevant fields from the file i.e. the sample array, GT and chromosome variants

{'samples': array(['HG00096', 'HG00097', 'HG00098', ..., 'NA21142', 'NA21143',
        'NA21144'], dtype=object), 'calldata/GT': array([[[ 0,  0],
         [ 0,  0],
         [ 0,  0],
         ...,
         [ 0,  0],
         [ 0,  0],
         [ 0,  0]],
 
        [[ 0,  0],
         [-1, -1],
         [ 0,  0],
         ...,
         [ 0,  0],
         [ 0,  0],
         [ 0,  0]],
 
        [[ 0,  0],
         [ 0,  0],
         [ 0,  0],
         ...,
         [ 0,  0],
         [ 0,  1],
         [ 0,  1]],
 
        ...,
 
        [[ 0, -1],
         [ 0, -1],
         [ 0, -1],
         ...,
         [ 0, -1],
         [ 0, -1],
         [ 0, -1]],
 
        [[ 1, -1],
         [ 1, -1],
         [ 1, -1],
         ...,
         [ 1, -1],
         [ 1, -1],
         [ 1, -1]],
 
        [[ 0, -1],
         [ 0, -1],
         [ 0, -1],
         ...,
         [ 0, -1],
         [ 0, -1],
         [ 0, -1]]], dtype=int8), 'variants/CHROM': array(['1', '1', '1', ..., 'MT', 'MT', 'M

In [8]:
# TO get a rough estimate of the run time of the program except the loading part
start_time=time.time()

In [9]:
chrom_data=data['variants/CHROM']
gt_data=data['calldata/GT']
samples_data=data['samples']

In [10]:
print(type(chrom_data),type(gt_data),type(samples_data))

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [11]:
gendata = allel.GenotypeChunkedArray(data['calldata/GT'])

In [12]:
gendata

<GenotypeChunkedArray shape=(905788, 3450, 2) dtype=int8
   nbytes=5.8G
   values=numpy.ndarray>

In [13]:
#To get the frequency of each of the chromosome variants and they are presented in a sorted order so makes it easy to access it
import collections
indices=dict(collections.Counter(chrom_data))

In [14]:
#Function to get the start and end indices of a particular chromosome variant

def get_indices(indices,samp):    
    start_ind=sum(list(indices.values())[:list(indices.keys()).index(samp)])
    end_ind=start_ind+indices[samp]
    return start_ind,end_ind

In [15]:
test_chrom='1' # Specify chromosome which needs to be analysed

start_index,end_index=get_indices(indices,test_chrom) 
print(start_index,end_index)

0 71302


In [16]:
gendata_for_chrom=gendata[start_index:end_index]

In [17]:
gendata_for_chrom

<GenotypeArray shape=(71302, 3450, 2) dtype=int8>
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 ./. 0/0 0/0 0/0 ... 0/0 ./. 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/1 0/0 0/1 0/1
...
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

In [18]:
# Analysing the data to check the alleles distribution in the data
count_all = gendata_for_chrom.count_alleles()[:]

In [19]:
count_all

<AlleleCountsArray shape=(71302, 2) dtype=int32>
6797   73
6205  117
6619  241
...
4938 1958
4978 1908
6754  146

In [20]:
#Removing/ Filtering out all the singletons present in the data i.e. though biallelic singletons dont exist 

filter_out = (count_all.max_allele() == 1) & (count_all[:, :2].min(axis=1) > 1)
gf = gendata_for_chrom.compress(filter_out, axis=0)
gf

<GenotypeArray shape=(71299, 3450, 2) dtype=int8>
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 ./. 0/0 0/0 0/0 ... 0/0 ./. 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/1 0/0 0/1 0/1
...
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

In [23]:
# Convert data to a 2-D matrix where each cell has the number of non-reference alleles per call.
gn = gf.to_n_alt()
gn.shape

(71299, 3450)

In [24]:
gn

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

One assumption made before doing PCA is that the data features is independent of each other. 
Usually PCA works best when the data is iid but not really sure till what extent is it relevant to 
this data. Something which needs to be looked into

In [25]:
# The allel.pca standardizes the data internally using the patterson algorithm so no need of doing it separately

allel_pca_time=time.time()
coords1, model1 = allel.pca(gn, 100, scaler='patterson')
print('TIME TAKEN FOR ALLEL PCA: ', time.time()-allel_pca_time)

TIME TAKEN FOR ALLEL PCA:  48.97733736038208


In [26]:
# The number of components can be changed accordingly but i choose to use the scikit PCA as it allows a component 
# for preserving the eigen energy of the matrix.I have tried using both of them to see the results.
# and to check the differences between them

#Standardizing the data and feeding it to the PCA                    
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scikit_pca_time=time.time()
gn=StandardScaler().fit_transform(gn)
pca = PCA(0.99)
gn_pca=pca.fit_transform(gn.T)
print('TIME TAKEN FOR SCIKIT PCA: ',time.time()-scikit_pca_time)

/media/ssd2/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/media/ssd2/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


TIME TAKEN FOR SCIKIT PCA:  45.55870079994202


In [27]:
#Analysis of scikit PCA ..rough estimate about the distribution
print(pca.explained_variance_ratio_, pca.singular_values_)
print(gn_pca.shape)

[1.01047262e-01 3.88385247e-02 1.12504809e-02 ... 4.13035871e-05
 4.12971756e-05 4.11970124e-05] [3607.13847843 2236.30947198 1203.61082249 ...   72.92802059   72.92236007
   72.83387255]
(3450, 3141)


In [28]:
coords1.shape

(3450, 100)

In [29]:
import umap

Using the defualt parameters of UMAP as of now. 

In [ ]:
#Applying UMAP on both the obtained PCA matrices
#Need to do something like grid search over the parameters of UMAP to find the best ones.

umap_time=time.time()

red_1=umap.UMAP(random_state=42).fit_transform(gn_pca)
red_2=umap.UMAP(random_state=42).fit_transform(coords1)

print('TIME TAKEN FOR UMAP: ',(time.time()-umap_time)/2)

In [31]:
red_1.shape, red_2.shape

((3450, 2), (3450, 2))

In [32]:
red_1

array([[-5.1662517, -5.4382834],
       [-5.6254086, -5.7337685],
       [-4.5899796, -4.869696 ],
       ...,
       [ 5.2080507, -9.802207 ],
       [ 5.1593947, -9.833728 ],
       [ 5.3819013, -9.908462 ]], dtype=float32)

In [33]:
red_2

array([[  3.8394535 , -12.697194  ],
       [  3.58669   , -12.903686  ],
       [  3.5765128 , -13.158334  ],
       ...,
       [ 16.201605  ,   0.82686484],
       [ 15.870831  ,   0.8836586 ],
       [ 16.307354  ,   1.1136986 ]], dtype=float32)

In [35]:
# Saving the reduced data(2 dimensional) in the numpy format
import numpy as np
np.save('{}_scikit_pca.npy'.format(test_chrom),red_1)
np.save('{}_allel_pca.npy'.format(test_chrom),red_2)

In [36]:
#Reading the data from panel and tsv file to visualize
import pandas as pd

panel_file= pd.read_csv('affy_samples.20141118.panel', sep="\t", header=0)
population_file = pd.read_csv('20131219.populations.tsv', sep="\t", header=0)

In [37]:
population_file

,Population Description,Population Code,Super Population,DNA from Blood,Offspring available from trios,Pilot Samples,Phase1 Samples,Final Phase Samples,Total
0,"Chinese Dai in Xishuangbanna, China",CDX,EAS,no,yes,0.0,0.0,99.0,99.0
1,"Han Chinese in Bejing, China",CHB,EAS,no,no,91.0,97.0,103.0,106.0
2,"Japanese in Tokyo, Japan",JPT,EAS,no,no,94.0,89.0,104.0,105.0
3,"Kinh in Ho Chi Minh City, Vietnam",KHV,EAS,yes,yes,0.0,0.0,101.0,101.0
4,"Southern Han Chinese, China",CHS,EAS,no,yes,0.0,100.0,108.0,112.0
5,Bengali in Bangladesh,BEB,SAS,no,yes,0.0,0.0,86.0,86.0
6,"Gujarati Indian in Houston,TX",GIH,SAS,no,yes,0.0,0.0,106.0,106.0
7,Indian Telugu in the UK,ITU,SAS,yes,yes,0.0,0.0,103.0,103.0
8,"Punjabi in Lahore,Pakistan",PJL,SAS,yes,yes,0.0,0.0,96.0,96.0
9,Sri Lankan Tamil in the UK,STU,SAS,yes,yes,0.0,0.0,103.0,103.0


In [38]:
population_file.drop([26,27,28])

,Population Description,Population Code,Super Population,DNA from Blood,Offspring available from trios,Pilot Samples,Phase1 Samples,Final Phase Samples,Total
0,"Chinese Dai in Xishuangbanna, China",CDX,EAS,no,yes,0.0,0.0,99.0,99.0
1,"Han Chinese in Bejing, China",CHB,EAS,no,no,91.0,97.0,103.0,106.0
2,"Japanese in Tokyo, Japan",JPT,EAS,no,no,94.0,89.0,104.0,105.0
3,"Kinh in Ho Chi Minh City, Vietnam",KHV,EAS,yes,yes,0.0,0.0,101.0,101.0
4,"Southern Han Chinese, China",CHS,EAS,no,yes,0.0,100.0,108.0,112.0
5,Bengali in Bangladesh,BEB,SAS,no,yes,0.0,0.0,86.0,86.0
6,"Gujarati Indian in Houston,TX",GIH,SAS,no,yes,0.0,0.0,106.0,106.0
7,Indian Telugu in the UK,ITU,SAS,yes,yes,0.0,0.0,103.0,103.0
8,"Punjabi in Lahore,Pakistan",PJL,SAS,yes,yes,0.0,0.0,96.0,96.0
9,Sri Lankan Tamil in the UK,STU,SAS,yes,yes,0.0,0.0,103.0,103.0


In [39]:
panel_file

,sample,pop,inphase3
0,HG00096,GBR,1
1,HG00097,GBR,1
2,HG00098,GBR,0
3,HG00099,GBR,1
4,HG00100,GBR,1
5,HG00101,GBR,1
6,HG00102,GBR,1
7,HG00104,GBR,0
8,HG00105,GBR,1
9,HG00106,GBR,1


In [40]:
temp=population_file['Population Code'].tolist()
full_form=population_file['Population Description'].tolist()

final=[]
for i in panel_file['pop']:
    final.append(full_form[temp.index(i)])

In [41]:
final

['British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Sco

In [42]:
#Appending the 2 dimensions and full forms to a single dataframe

#red_1 = scikit pca + umap
#red_2 = allec pca + umap 

panel_file['x1']=red_1[:,0]
panel_file['x2']=red_1[:,1]
panel_file['full_form']=final

In [43]:
panel_file

,sample,pop,inphase3,x1,x2,full_form
0,HG00096,GBR,1,-5.166252,-5.438283,British in England and Scotland
1,HG00097,GBR,1,-5.625409,-5.733768,British in England and Scotland
2,HG00098,GBR,0,-4.589980,-4.869696,British in England and Scotland
3,HG00099,GBR,1,-5.061214,-5.806071,British in England and Scotland
4,HG00100,GBR,1,-4.883696,-5.468121,British in England and Scotland
5,HG00101,GBR,1,-4.559065,-5.489052,British in England and Scotland
6,HG00102,GBR,1,-5.246717,-5.970696,British in England and Scotland
7,HG00104,GBR,0,-5.150257,-5.734216,British in England and Scotland
8,HG00105,GBR,1,-4.560771,-5.117169,British in England and Scotland
9,HG00106,GBR,1,-4.546440,-5.471063,British in England and Scotland


Visualizing via Plotly 

In [44]:
import plotly
plotly.__version__

'3.7.0'

In [45]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import numpy as np
import cufflinks as cf

In [46]:
init_notebook_mode(connected=True)

The plots are interactive. Helps in better analysis

In [47]:
#Unlabelled plot in 2 D
iplot(panel_file.iplot(asFigure=True,
                               kind='scatter',x='x1',y='x2', xTitle='x1',yTitle='x2', text='full_form',title='2-D plot',mode='markers'))

In [48]:
pop_codes=set(panel_file['pop'].tolist())

In [49]:
pop_codes

{'ACB',
 'ASW',
 'BEB',
 'CDX',
 'CEU',
 'CHB',
 'CHS',
 'CLM',
 'ESN',
 'FIN',
 'GBR',
 'GIH',
 'GWD',
 'IBS',
 'ITU',
 'JPT',
 'KHV',
 'LWK',
 'MSL',
 'MXL',
 'PEL',
 'PJL',
 'PUR',
 'STU',
 'TSI',
 'YRI'}

In [50]:
#Labelled plot 2-D

fig = {
    'data': [
        {'x': panel_file.loc[panel_file['pop']==i]['x1'], 'y': panel_file.loc[panel_file['pop']==i]['x2'], 'text': panel_file.loc[panel_file['pop']==i]['full_form'] , 'mode': 'markers', 'name': i}
        for i in pop_codes
    ],
    'layout': {
        'xaxis': {'title': 'x1'},
        'yaxis': {'title': 'x2'}
    }
}
iplot(fig, filename='scatter-plot')

In [51]:
time.time()-start_time

673.8710880279541

References:

    
https://umap-learn.readthedocs.io/en/latest/index.html

https://scikit-allel.readthedocs.io/en/stable/

https://scikit-learn.org/stable/

https://plot.ly/


https://www.wikipedia.org/

https://www.youtube.com/channel/UCKbkfKk65PZyRCzUwXOJung

